In [26]:
import io
import requests
import torch
import torch.nn as nn
#from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import models, transforms
from torch.autograd import Variable
from utils.timer import Timer
import os
import glob
%matplotlib inline

In [27]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.device_count() #Count GPU

2

In [28]:
dtype = torch.cuda.FloatTensor
resnet101=models.resnet101(pretrained=True) 
resnet101.cuda()
resnet101 = nn.DataParallel(resnet101)
resnet101.to(device)
resnet101.eval()

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): C

In [29]:
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
   transforms.Resize(256),
   transforms.CenterCrop(224),
   transforms.ToTensor(),
   normalize
])

In [ ]:
LABELS_URL = 'https://s3.amazonaws.com/outcome-blog/imagenet/labels.json'
labels = {int(key):value for (key, value)
          in requests.get(LABELS_URL).json().items()}

In [30]:
#12*9 = 108 tiles
#mosiac size (2688,2016)
img_pil = Image.open('../data/mosaik30.jpg')

In [31]:
def crop(im,height,width):
    imgwidth, imgheight = im.size
    stepsize = 224
    for i in range(0,imgheight,224):
        for j in range(0,imgwidth,224):
            #print (i,j)
            box = (j,j+stepsize,i, i+stepsize)
            yield im.crop(box)

crop(img_pil,224,224)

<generator object crop at 0x2aef2c266c50>

In [33]:
height=224
width=224
start_num = 0

path = os.path.abspath('/home/aishwas/Stream/ResNet101_Classification/split_data/')
file_name = os.path.basename('/home/aishwas/Stream/data/mosaik30.jpg')
file = os.path.splitext(file_name)[0]

timer = Timer()
timer.tic()

for k,piece in enumerate(crop(img_pil,height,width),start_num):
    img=Image.new('RGB',(width,height), 224)
    img.paste(piece)
    newfile = '/home/aishwas/Stream/ResNet101_Classification/split_data/test.jpg'
    img.save(newfile)
    testimage = Image.open(newfile)
    img_tensor = preprocess(testimage)
    img_tensor.unsqueeze_(0)
    img_variable = Variable(img_tensor)
    out = resnet101(img_variable.cuda())
    print("Image {} is classified as {}".format(file + "_" + str(k) + ".jpg",labels[out.data.cpu().numpy().argmax()]))

timer.toc()

Image mosaik30_0.jpg is classified as red wine
Image mosaik30_1.jpg is classified as red wine
Image mosaik30_2.jpg is classified as red wine
Image mosaik30_3.jpg is classified as red wine
Image mosaik30_4.jpg is classified as red wine
Image mosaik30_5.jpg is classified as red wine
Image mosaik30_6.jpg is classified as red wine
Image mosaik30_7.jpg is classified as red wine
Image mosaik30_8.jpg is classified as red wine
Image mosaik30_9.jpg is classified as red wine
Image mosaik30_10.jpg is classified as red wine
Image mosaik30_11.jpg is classified as red wine
Image mosaik30_12.jpg is classified as worm fence, snake fence, snake-rail fence, Virginia fence
Image mosaik30_13.jpg is classified as red wine
Image mosaik30_14.jpg is classified as red wine
Image mosaik30_15.jpg is classified as red wine
Image mosaik30_16.jpg is classified as red wine
Image mosaik30_17.jpg is classified as red wine
Image mosaik30_18.jpg is classified as red wine
Image mosaik30_19.jpg is classified as red wine
I

5.089181900024414

In [25]:
print("Time for Image classification {:.3f}s ".format(timer.total_time()))

Time for Image classification 4.139s 
